In [32]:
from pprint import pprint
from datasets import load_dataset
from transformers import BatchEncoding, AutoTokenizer
from tqdm import tqdm

In [33]:
train_dataset = load_dataset('llm-book/JGLUE', name='JNLI', split='train')
valid_dataset = load_dataset('llm-book/JGLUE', name='JNLI', split='validation')

In [34]:
train_dataset[0]

{'sentence_pair_id': '0',
 'yjcaptions_id': '100124-104404-104405',
 'sentence1': '二人の男性がジャンボジェット機を見ています。',
 'sentence2': '2人の男性が、白い飛行機を眺めています。',
 'label': 2}

In [35]:
print(train_dataset.features["label"])
print(train_dataset[1])

ClassLabel(names=['entailment', 'contradiction', 'neutral'], id=None)
{'sentence_pair_id': '1', 'yjcaptions_id': '100124-104405-104404', 'sentence1': '2人の男性が、白い飛行機を眺めています。', 'sentence2': '二人の男性がジャンボジェット機を見ています。', 'label': 2}


In [36]:
train_dataset

Dataset({
    features: ['sentence_pair_id', 'yjcaptions_id', 'sentence1', 'sentence2', 'label'],
    num_rows: 20073
})

In [37]:
transformer_model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(transformer_model_name)

In [38]:
# for row in tqdm(valid_dataset):
#     print(row["sentence1"])

In [39]:
from torch.utils.data import IterableDataset, Dataset
from tqdm import tqdm

class Dataset1(IterableDataset):
    def __init__(self, ds):
        self.features = [
            {
                'sentence_pair_id': row['sentence_pair_id'],
                'yjcaptions_id': row['yjcaptions_id'],
                'sentence1': row['sentence1'],
                'sentence2': row['sentence2'],
                'label': row['label']
            } for row in tqdm(ds)
        ]

    def __len__(self):
        return len(self.features)

    def __iter__(self):
        return iter(self.features)

train_dataset1 = Dataset1(train_dataset)
valid_dataset1 = Dataset1(valid_dataset)













100%|██████████| 20073/20073 [00:01<00:00, 16826.86it/s]


100%|██████████| 2434/2434 [00:00<00:00, 14818.15it/s]


In [40]:
# datasetのサイズを確認
print(len(train_dataset1))
# datasetの中身を確認
tmp = next(iter(train_dataset1))
print(tmp)

tmp = next(iter(train_dataset))
print(tmp)

20073
{'sentence_pair_id': '0', 'yjcaptions_id': '100124-104404-104405', 'sentence1': '二人の男性がジャンボジェット機を見ています。', 'sentence2': '2人の男性が、白い飛行機を眺めています。', 'label': 2}
{'sentence_pair_id': '0', 'yjcaptions_id': '100124-104404-104405', 'sentence1': '二人の男性がジャンボジェット機を見ています。', 'sentence2': '2人の男性が、白い飛行機を眺めています。', 'label': 2}


In [41]:
list(train_dataset1)[0:4]

[{'sentence_pair_id': '0',
  'yjcaptions_id': '100124-104404-104405',
  'sentence1': '二人の男性がジャンボジェット機を見ています。',
  'sentence2': '2人の男性が、白い飛行機を眺めています。',
  'label': 2},
 {'sentence_pair_id': '1',
  'yjcaptions_id': '100124-104405-104404',
  'sentence1': '2人の男性が、白い飛行機を眺めています。',
  'sentence2': '二人の男性がジャンボジェット機を見ています。',
  'label': 2},
 {'sentence_pair_id': '2',
  'yjcaptions_id': '100142-104431-104432',
  'sentence1': '男性が子供を抱き上げて立っています。',
  'sentence2': '坊主頭の男性が子供を抱いて立っています。',
  'label': 2},
 {'sentence_pair_id': '3',
  'yjcaptions_id': '100142-104432-104431',
  'sentence1': '坊主頭の男性が子供を抱いて立っています。',
  'sentence2': '男性が子供を抱き上げて立っています。',
  'label': 0}]

In [42]:
import torch
from transformers import AutoTokenizer

class DataCollator1():
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, examples):
        # sentence1とsentence2を連結し、encodingsを返す
        examples = {
            'sentence1': list(map(lambda x: x['sentence1'], examples)),
            'sentence2': list(map(lambda x: x['sentence2'], examples)),
            'label': list(map(lambda x: x['label'], examples)),
        }
        encodings = self.tokenizer(
                                   examples['sentence1'],
                                   examples['sentence2'],
                                   padding=True, 
                                   truncation=True,
                                   max_length=self.max_length,
                                   return_tensors='pt')

        encodings['labels'] = torch.tensor(examples['label'])
        return encodings


tokenizer = AutoTokenizer.from_pretrained(transformer_model_name)
data_collator = DataCollator1(tokenizer)

In [43]:
from torch.utils.data import DataLoader
loader = DataLoader(train_dataset1, collate_fn=data_collator, batch_size=8)
batch = next(iter(loader))
# batch = next(iter(loader))

# batchの各keyのsizeを確認
for k, v in batch.items():
    print(k, v.size())

# batchの中身を確認
pprint(batch["input_ids"][3])
print(batch["labels"])

input_ids torch.Size([8, 36])
token_type_ids torch.Size([8, 36])
attention_mask torch.Size([8, 36])
labels torch.Size([8])
tensor([    2, 27714,  6589,   464, 13341,   430, 13275,   500, 16563,   456,
        16996,   456,   422, 12995,   385,     3, 13341,   430, 13275,   500,
        18967, 12867,   456, 16996,   456,   422, 12995,   385,     3,     0,
            0,     0,     0,     0,     0,     0])
tensor([2, 2, 2, 0, 2, 2, 2, 0])


In [44]:
train_dataset.features["label"]

ClassLabel(names=['entailment', 'contradiction', 'neutral'], id=None)

In [45]:
from transformers import AutoModelForSequenceClassification

class_label = train_dataset.features["label"]
label2id = {label: id for id, label in enumerate(class_label.names)}
id2label = {id: label for id, label in enumerate(class_label.names)}
model = AutoModelForSequenceClassification.from_pretrained(
    transformer_model_name,
    num_labels=class_label.num_classes,
    label2id=label2id,  # ラベル名からIDへの対応を指定
    id2label=id2label,  # IDからラベル名への対応を指定
)
print(type(model).__name__)

# モデルの出力を確認
outputs = model(**batch)
outputs

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification


SequenceClassifierOutput(loss=tensor(0.8577, grad_fn=<NllLossBackward0>), logits=tensor([[-0.2862, -0.1456,  0.5988],
        [-0.2874, -0.1582,  0.6100],
        [-0.2977, -0.1579,  0.5402],
        [-0.2814, -0.1529,  0.5309],
        [-0.2276, -0.1309,  0.5875],
        [-0.2257, -0.1272,  0.5945],
        [-0.2463, -0.1110,  0.5373],
        [-0.2219, -0.0948,  0.5658]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [46]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="output_jnli",  # 結果の保存フォルダ
    per_device_train_batch_size=8,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=8,  # 評価時のバッチサイズ
    learning_rate=2e-5,  # 学習率
    lr_scheduler_type="linear",  # 学習率スケジューラの種類
    warmup_ratio=0.1,  # 学習率のウォームアップの長さを指定
    num_train_epochs=3,  # エポック数
    label_names=['labels'],  # ラベル名を指定
    save_strategy="epoch",  # チェックポイントの保存タイミング
    logging_strategy="epoch",  # ロギングのタイミング
    evaluation_strategy="epoch",  # 検証セットによる評価のタイミング
    load_best_model_at_end=True,  # 訓練後に開発セットで最良のモデルをロード
    metric_for_best_model="accuracy",  # 最良のモデルを決定する評価指標
    fp16=True,  # 自動混合精度演算の有効化
    remove_unused_columns=False, # 入力データに含まれない列を削除するかどうか(https://qiita.com/m__k/items/2c4e476d7ac81a3a44af)
)

# training_args = TrainingArguments(
#     output_dir='./output/model',
#     evaluation_strategy='epoch',    # 検証セットによる評価のタイミング
#     logging_strategy='epoch',   # ロギングのタイミング
#     save_strategy='epoch',  # チェックポイントの保存タイミング
#     # save_total_limit=1,
#     label_names=['labels'], # ラベル名を指定
#     lr_scheduler_type='linear',   # 学習率スケジューラの種類
#     metric_for_best_model='accuracy',   # 最良のモデルを決定する評価指標
#     load_best_model_at_end=True,    # 訓練後に開発セットで最良のモデルをロード
#     per_device_train_batch_size=16, # 訓練時のバッチサイズ
#     per_device_eval_batch_size=16,  # 評価時のバッチサイズ
#     num_train_epochs=5,
#     # remove_unused_columns=False,
#     # report_to='none'
#     fp16=True,  # 自動混合精度演算の有効化
# )


In [47]:
import numpy as np

def compute_accuracy(
    eval_pred: tuple[np.ndarray, np.ndarray]
) -> dict[str, float]:
    """予測ラベルと正解ラベルから正解率を計算"""
    predictions, labels = eval_pred
    # predictionsは各ラベルについてのスコア
    # 最もスコアの高いインデックスを予測ラベルとする
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [48]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=list(train_dataset1),
    eval_dataset=list(valid_dataset1),
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_accuracy,
)
# trainer.train()
trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])

  0%|          | 0/7530 [00:00<?, ?it/s]

{'loss': 0.4928, 'learning_rate': 1.4826619448133392e-05, 'epoch': 1.0}


  0%|          | 0/305 [00:00<?, ?it/s]

{'eval_loss': 0.47213637828826904, 'eval_accuracy': 0.866885784716516, 'eval_runtime': 5.225, 'eval_samples_per_second': 465.837, 'eval_steps_per_second': 58.373, 'epoch': 1.0}
{'loss': 0.27, 'learning_rate': 7.42216319905563e-06, 'epoch': 2.0}


  0%|          | 0/305 [00:00<?, ?it/s]

{'eval_loss': 0.42057183384895325, 'eval_accuracy': 0.9018077239112572, 'eval_runtime': 4.4595, 'eval_samples_per_second': 545.8, 'eval_steps_per_second': 68.393, 'epoch': 2.0}
{'loss': 0.1439, 'learning_rate': 2.06581083075107e-08, 'epoch': 3.0}


  0%|          | 0/305 [00:00<?, ?it/s]

{'eval_loss': 0.5329641103744507, 'eval_accuracy': 0.900164338537387, 'eval_runtime': 4.3969, 'eval_samples_per_second': 553.573, 'eval_steps_per_second': 69.367, 'epoch': 3.0}
{'train_runtime': 695.4093, 'train_samples_per_second': 86.595, 'train_steps_per_second': 10.828, 'train_loss': 0.30223845827626994, 'epoch': 3.0}


TrainOutput(global_step=7530, training_loss=0.30223845827626994, metrics={'train_runtime': 695.4093, 'train_samples_per_second': 86.595, 'train_steps_per_second': 10.828, 'train_loss': 0.30223845827626994, 'epoch': 3.0})

In [50]:
# 検証セットでモデルを評価
eval_metrics = trainer.evaluate(valid_dataset)
pprint(eval_metrics)

  0%|          | 0/305 [00:00<?, ?it/s]

{'epoch': 3.0,
 'eval_accuracy': 0.9018077239112572,
 'eval_loss': 0.42057183384895325,
 'eval_runtime': 5.1817,
 'eval_samples_per_second': 469.73,
 'eval_steps_per_second': 58.861}
